In [1]:
from paperqa import Docs, Settings

In [2]:
import json
import sys

import os
#print current working directory
print("Current working directory:", os.getcwd())
#change to the directory where the source located
os.chdir("../Reproduction/Pipeline_reproduction/src")
print("Current working directory:", os.getcwd())

from MC_gpt import ScientificQAEvaluator
import nest_asyncio
from typing import Optional

Current working directory: /Users/apple/Documents/GitLab_Projects/master_project/xx823/notebooks
Current working directory: /Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/Pipeline_reproduction/src


In [5]:
with open("../../../data/Questions/formatted_questions_test/questions.json", "r") as f:
    questions = json.load(f)

In [6]:
print(questions)

[{'question': 'Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?', 'choices': ['(A) 41%', '(B) 51%', '(C) 31%', '(D) Insufficient information to answer the question', '(E) 11%', '(F) 21%'], 'correct_answer': 'C', 'unsure_option': 'D', 'sources': ['https://doi.org/10.1038/s41467-024-44782-6'], 'ideal': '31%', 'distractors': ['21%', '11%', '41%', '51%']}, {'question': 'At least how long do SynNotch-MCF10DCIS cells express BFP after contact with GFP+BMSC3 cells?', 'choices': ['(A) 0 h', '(B) 6 h', '(C) 24', '(D) 12 h', '(E) 48 h', '(F) 96 h', '(G) Insufficient information to answer the question', '(H) 72 h'], 'correct_answer': 'H', 'unsure_option': 'G', 'sources': ['https://doi.org/10.1073/pnas.2322688121'], 'ideal': '72 h', 'distractors': ['24', '48 h', '0 h', '12 h', '6 h', '96 h']}, {'question': 'DK015 and DK038 strains of Verticillium dahliae have in common approximately what 

In [ ]:
def format_multiple_choice_question(question, choices):
    formatted = f"Question:Q: {question}; Options: "
    formatted += " ".join(choices)
    return formatted


In [ ]:
formatted_q=[]
for q in questions:
    formatted_q.append(format_multiple_choice_question(q["question"], q["choices"]))

## hard code pipeline wihout agents

In [ ]:
from paperqa.agents.search import get_directory_index


In [ ]:
from paperqa.settings import Settings, AgentSettings

In [ ]:
import os

In [ ]:
class PaperQAClient:
    """A singleton client for PaperQA2 that builds the index once and reuses it."""
    
    _instance = None
    _docs = None
    _index_built = False
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super(PaperQAClient, cls).__new__(cls)
        return cls._instance
    
    def __init__(self, settings: Optional[Settings] = None):
        # Set up the settings
        self.paper_directory = '../../../data/papers'
        if settings is None:
            self.settings = Settings(
                temperature=0.1,
                llm='gpt-4o-mini',
                paper_directory=self.paper_directory
            )
        else:
            self.settings = settings
            self.paper_directory = settings.paper_directory
        
        # Apply nest_asyncio
        nest_asyncio.apply()
        
        # Initialize docs but don't load papers yet (that happens in init_docs)
        if self._docs is None:
            self._docs = Docs(index_path=self.paper_directory)
    
    async def init_docs(self):
        """Initialize documents asynchronously (must be called before first query)"""
        if not self._docs or len(self._docs.docs) == 0:
            # Add the pdf files in the doc_paths
            doc_paths = [os.path.join(self.paper_directory, doc) 
                        for doc in os.listdir(self.paper_directory) 
                        if doc.endswith('.pdf')]
            
            for doc_path in doc_paths:
                try:
                    await self._docs.aadd(doc_path)
                except Exception as e:
                    print(f"Error loading document {doc_path}: {e}")
    
    async def _build_index_if_needed(self):
        """Build the document index if not already built"""
        if not self._index_built:
            print("Building PaperQA2 document index (only happens once)...")
            await get_directory_index(settings=self.settings)
            self._index_built = True
    
    async def aask(self, question: str) -> str:
        """Async method to query PaperQA2"""
        # Initialize docs if needed
        await self.init_docs()
        
        # Ensure index is built
        await self._build_index_if_needed()
        
        # Query PaperQA2
        response_pqa = await self._docs.aquery(question, settings=self.settings)
        
        # Return the answer
        return response_pqa.answer

    def ask(self, question: str) -> str:
        """Synchronous method to query PaperQA2"""
        import asyncio
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(self.aask(question))



In [ ]:
pqa_client = PaperQAClient(
        settings=Settings(
            llm="gpt-4o-mini",
            temperature=0.1,
            paper_directory="../../../data/papers"
        )
    )
    

In [ ]:
grader_input_data = []

In [ ]:
total_questions = len(formatted_q)
for i, q in enumerate(formatted_q): # test for 49 questions
    current_question_num = i + 1
    print(f"Processing question {current_question_num}/{total_questions}: {q[:50]}...")
    
    # Get PaperQA2's response for this question
    response = pqa_client.ask(q)
    
    # Log completion status
    print(f"Completed question {current_question_num}/{total_questions}, {total_questions - current_question_num} remaining")
    
    # Add the question and response to the grader input
    grader_input_data.append({
        "question":questions[i]["question"],
        "choices": questions[i]["choices"],
        "response": response,
        "correct_answer": questions[i]["correct_answer"],
        "unsure_option": questions[i]["unsure_option"]
    })

Processing question 1/49: Question:Q: Approximately what percentage of topol...


15:40:04 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
15:40:04 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
15:40:04 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
15:40:04 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


Error loading document /Users/apple/Documents/GitLab_Projects/master_project/xx823/papers/RNA damage compartmentalization by DHX9 stress granules.pdf: litellm.NotFoundError: OpenAIException - Project `proj_8WO6cHG6Ics15ycd35qOE2RY` does not have access to model `gpt-4o-2024-11-20`. Received Model Group=gpt-4o-2024-11-20
Available Model Group Fallbacks=None


15:40:06 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
15:40:06 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
15:40:06 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
15:40:06 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


Error loading document /Users/apple/Documents/GitLab_Projects/master_project/xx823/papers/doddihal-et-al-2024-a-pak-family-kinase-and-the-hippo-yorkie-pathway-modulate-wnt-signaling-to-functionally-integrate.pdf: litellm.NotFoundError: OpenAIException - Project `proj_8WO6cHG6Ics15ycd35qOE2RY` does not have access to model `gpt-4o-2024-11-20`. Received Model Group=gpt-4o-2024-11-20
Available Model Group Fallbacks=None


...

Error loading document /Users/apple/Documents/GitLab_Projects/master_project/xx823/papers/geller-et-al-2024-identification-of-type-vi-secretion-system-effector-immunity-pairs-using-structural-bioinformatics.pdf: litellm.NotFoundError: OpenAIException - Project `proj_8WO6cHG6Ics15ycd35qOE2RY` does not have access to model `gpt-4o-2024-11-20`. Received Model Group=gpt-4o-2024-11-20
Available Model Group Fallbacks=None


16:30:50 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
16:30:50 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
16:30:50 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
16:30:50 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


Error loading document /Users/apple/Documents/GitLab_Projects/master_project/xx823/papers/Leveraging chromatin state transitions for the identification of regulatory networks orchestrating heart regeneration.pdf: litellm.NotFoundError: OpenAIException - Project `proj_8WO6cHG6Ics15ycd35qOE2RY` does not have access to model `gpt-4o-2024-11-20`. Received Model Group=gpt-4o-2024-11-20
Available Model Group Fallbacks=None


16:30:50 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
16:30:50 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
16:30:50 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
16:30:50 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


Completed question 49/49, 0 remaining


In [ ]:
import json

In [ ]:
with open("../paperqa_response/grader_input_data_test_4omini_no_agent.json", "w") as f:
    json.dump(grader_input_data, f, indent=2)

## Now, Use AG2 to eval

In [ ]:
with open("../paperqa_response/grader_input_data_test_4omini_no_agent.json", "r") as f:
    loaded_data = json.load(f)

In [ ]:
evaluator = ScientificQAEvaluator(model_name="gpt-4o-mini")

In [ ]:
evaluator.process_questions(questions)


Processing question 1/49
UserProxy (to ScientificMCAgent):

Please answer this multiple-choice question: Question: Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?; Options: (A) 41% (B) 51% (C) 31% (D) Insufficient information to answer the question (E) 11% (F) 21%

--------------------------------------------------------------------------------
ScientificMCAgent (to UserProxy):

To answer the question regarding the percentage of topologically associated domains (TADs) in the GM12878 blood cell line that DiffDomain classifies as reorganized in the K562 cell line, we need to consider the specific findings from studies that have analyzed TADs across different cell types.

The options provided are specific percentages, and without access to the exact data or study results, it is challenging to determine the correct answer definitively. However, studies in this area often report 

In [ ]:
evaluator.display_metrics()


===== EVALUATION METRICS =====
Total questions: 49
Correct answers: 8 (16.33%)
Incorrect answers: 24 (48.98%)
Unsure answers: 17 (34.69%)
Manual review needed: 0 (0.00%)
Precision: 25.00%
Accuracy: 16.33%


In [ ]:
evaluator.save_results(experiment_name="GPT4o-mini_test_no_agent")


Results saved to evaluation_results_49MCs_gpt4o/GPT4o-mini_test_no_agent_20250506_163829_results.json
Metrics saved to evaluation_results_49MCs_gpt4o/GPT4o-mini_test_no_agent_20250506_163829_metrics.json


{'results_file': 'evaluation_results_49MCs_gpt4o/GPT4o-mini_test_no_agent_20250506_163829_results.json',
 'metrics_file': 'evaluation_results_49MCs_gpt4o/GPT4o-mini_test_no_agent_20250506_163829_metrics.json'}